In [1]:
import os
import random
import math

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import datasets, models, transforms
from torchvision.models.detection.retinanet import RetinaNet
from torchvision.models.detection.faster_rcnn import FasterRCNN
import  torchvision.transforms.functional as F

from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import xml.etree.ElementTree as ET
import collections
from torchvision.datasets.voc import VisionDataset

from functions import *
from functions_torch import *

In [2]:
params = {}
params['target_size']=(2000,1500)

DSRoots = ['/app/host/lacmus/dataset/'+d for d in
    ['full_lacmus_ds', 'spring_korolev_2019', 'summer_moscow_2019', 'summer_tambov_2019', 'winter_moscow_2018']]

In [3]:
# model = torchvision.models.detection.retinanet_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=False, 
#                                                            min_size=params['target_size'][0], max_size = params['target_size'][1])
# model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_LADD_epoch_8.pth'), strict=True)
# model = model.to(torch.device('cuda'))



In [4]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, num_classes=2, pretrained_backbone=True, 
                                                           min_size=params['target_size'][0], max_size = params['target_size'][1],
                                                           trainable_backbone_layers = 5)
model.load_state_dict(torch.load('/app/host/lacmus/weights/resnet50_FRCNN_LADD_epoch_9.pth'), strict=True)
model = model.to(torch.device('cuda'))

In [5]:
class LADDDataSET(torchvision.datasets.VisionDataset):
    def __init__(
            self,
            root: str,
            image_set: str,
            transforms: Optional[Callable] = None):     
        super(LADDDataSET, self).__init__(root, transforms=transforms)
        self.image_set = image_set

        voc_root = root
        image_dir = os.path.join(voc_root, 'JPEGImages')
        annotation_dir = os.path.join(voc_root, 'Annotations')

        if not os.path.isdir(voc_root):
            raise RuntimeError('Dataset not found or corrupted.')

        splits_dir = os.path.join(voc_root, 'ImageSets/Main')
        split_f = os.path.join(splits_dir, image_set.rstrip('\n') + '.txt')

        with open(os.path.join(split_f), "r") as f:
            file_names = [x.strip() for x in f.readlines()]

        self.images = [os.path.join(image_dir, x + ".jpg") for x in file_names]
        self.annotations = [os.path.join(annotation_dir, x + ".xml") for x in file_names]
        assert (len(self.images) == len(self.annotations))
        
    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is a dictionary of the XML tree.
        """
        img = Image.open(self.images[index]).convert('RGB')
        description = LADDDataSET.parse_voc_xml(
            ET.parse(self.annotations[index]).getroot())

        # get bounding box coordinates 
        num_objs = len(description['annotation']['object'])
        boxes = []
        for l in description['annotation']['object']:
            bb = l['bndbox']
            boxes.append([int(bb['xmin']), int(bb['ymin']), int(bb['xmax']), int(bb['ymax'])])

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)                
        target["labels"] = labels = torch.ones((num_objs,), dtype=torch.int64)  # there is only one class   
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    

    def __len__(self) -> int:
        return len(self.images)

    @staticmethod
    def parse_voc_xml(node: ET.Element) -> Dict[str, Any]:
        voc_dict: Dict[str, Any] = {}
        children = list(node)
        if children:
            def_dic: Dict[str, Any] = collections.defaultdict(list)
            for dc in map(LADDDataSET.parse_voc_xml, children):
                for ind, v in dc.items():
                    def_dic[ind].append(v)
            if node.tag == 'annotation':
                def_dic['object'] = [def_dic['object']]
            voc_dict = {
                node.tag:
                    {ind: v[0] if len(v) == 1 else v
                     for ind, v in def_dic.items()}
            }
        if node.text:
            text = node.text.strip()
            if not children:
                voc_dict[node.tag] = text
        return voc_dict

In [6]:
for ds in DSRoots:
    dataset = LADDDataSET(ds,'test',get_transform(train=False,target_size=params['target_size'])) 
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=False, num_workers=1,collate_fn=collate_fn)
    inference_res = evaluate(model,data_loader)
    print("Inference for %s, computing mAp : "%ds)
    print(evaluate_res(inference_res, iou_threshold = 0.5, score_threshold = 0.05))    
    print(evaluate_res(inference_res, iou_threshold = 0.6, score_threshold = 0.05))
    


Inference for /app/host/lacmus/dataset/full_lacmus_ds, computing mAp : 
0.9224656100089377
0.8816261173472251
Inference for /app/host/lacmus/dataset/spring_korolev_2019, computing mAp : 
0.7813762094212323
0.6792287784603297
Inference for /app/host/lacmus/dataset/summer_moscow_2019, computing mAp : 
0.8747487580439925
0.8296626896177549
Inference for /app/host/lacmus/dataset/summer_tambov_2019, computing mAp : 
0.9183631623027297
0.8801822715839946
Inference for /app/host/lacmus/dataset/winter_moscow_2018, computing mAp : 
0.9801216608440391
0.9686321758051379
